In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e3/sample_submission.csv
/kaggle/input/playground-series-s5e3/train.csv
/kaggle/input/playground-series-s5e3/test.csv


In [2]:
train_path = "/kaggle/input/playground-series-s5e3/train.csv"
test_path = "/kaggle/input/playground-series-s5e3/test.csv"

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

train_data.head()

,id,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall
0,0,1,1017.4,21.2,20.6,19.9,19.4,87.0,88.0,1.1,60.0,17.2,1
1,1,2,1019.5,16.2,16.9,15.8,15.4,95.0,91.0,0.0,50.0,21.9,1
2,2,3,1024.1,19.4,16.1,14.6,9.3,75.0,47.0,8.3,70.0,18.1,1
3,3,4,1013.4,18.1,17.8,16.9,16.8,95.0,95.0,0.0,60.0,35.6,1
4,4,5,1021.8,21.3,18.4,15.2,9.6,52.0,45.0,3.6,40.0,24.8,0


In [3]:
test_data.head()

,id,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed
0,2190,1,1019.5,17.5,15.8,12.7,14.9,96.0,99.0,0.0,50.0,24.3
1,2191,2,1016.5,17.5,16.5,15.8,15.1,97.0,99.0,0.0,50.0,35.3
2,2192,3,1023.9,11.2,10.4,9.4,8.9,86.0,96.0,0.0,40.0,16.9
3,2193,4,1022.9,20.6,17.3,15.2,9.5,75.0,45.0,7.1,20.0,50.6
4,2194,5,1022.2,16.1,13.8,6.4,4.3,68.0,49.0,9.2,20.0,19.4


In [4]:
#X = train_data.drop(columns=['rainfall', 'id', 'day']) 
X = train_data[['humidity', 'sunshine', 'dewpoint', 'pressure', 'cloud', 'temparature']]
y = train_data['rainfall'] 

In [5]:
from sklearn.model_selection import train_test_split

# Split the data: 80% for training, 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Видаляємо ті ж самі колонки з test_data
test_features = test_data.drop(columns=['id', 'day'])
test_features = test_features[X_train.columns]

# Check the size
print(X_train.shape, X_test.shape)


(1752, 6) (438, 6)


In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

for n in [50, 100, 200, 300]:
    
    model = RandomForestClassifier(n_estimators=n, max_depth=10, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"n_estimators: {n}, Accuracy: {accuracy:.4f}")

    

    importances = model.feature_importances_
    feature_names = X_train.columns



# Сортуємо та виводимо найважливіші фічі
sorted_features = sorted(zip(feature_names, importances), key=lambda x: x[1], reverse=True)
for feature, importance in sorted_features:
    print(f"{feature}: {importance:.4f}")

n_estimators: 50, Accuracy: 0.8539
n_estimators: 100, Accuracy: 0.8562
n_estimators: 200, Accuracy: 0.8493
n_estimators: 300, Accuracy: 0.8493
cloud: 0.3538
sunshine: 0.2303
humidity: 0.1374
dewpoint: 0.1001
temparature: 0.0918
pressure: 0.0866


In [7]:
#from sklearn.metrics import accuracy_score

#y_pred = model.predict(X_test)
#accuracy = accuracy_score(y_test, y_pred)

#print(f"n_estimators: {n}, Accuracy: {accuracy:.4f}")

In [8]:
print(test_features.dtypes)
print(X_train.dtypes)

humidity       float64
sunshine       float64
dewpoint       float64
pressure       float64
cloud          float64
temparature    float64
dtype: object
humidity       float64
sunshine       float64
dewpoint       float64
pressure       float64
cloud          float64
temparature    float64
dtype: object


In [9]:
print(test_features.isnull().sum())

humidity       0
sunshine       0
dewpoint       0
pressure       0
cloud          0
temparature    0
dtype: int64


In [10]:
test_features = test_features.fillna(test_features.mean())

In [11]:
# 1. Робимо передбачення для тестового набору Kaggle
test_predictions = model.predict(test_features)

# 2. Формуємо submission DataFrame
submission = pd.DataFrame({
    "id": test_data["id"],  # ID з тестового набору
    "rainfall": test_predictions  # Передбачені значення
})

# 3. Зберігаємо у CSV
submission.to_csv("submission.csv", index=False)
print("Submission file saved successfully!")

Submission file saved successfully!


In [12]:
!ls /kaggle/working/

__notebook__.ipynb  submission.csv


In [13]:
!head submission.csv

id,rainfall
2190,1
2191,1
2192,1
2193,0
2194,0
2195,1
2196,1
2197,1
2198,1
